In [201]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score

In [164]:
train_set=pd.read_csv("/content/Aviakompaniya/train_dataset.csv",index_col=0)
train_set.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
id,,,,,,,,,,,,,,,,,,,,,
1,Male,disloyal Customer,33,Business travel,Eco,571,2,3,2,4,...,4,3,1,3,4,3,4,10,3.0,0
2,Female,Loyal Customer,49,Business travel,Business,1431,4,1,4,4,...,5,5,5,5,3,5,3,0,0.0,1
3,Female,Loyal Customer,43,Business travel,Eco,867,1,4,4,4,...,1,1,1,1,1,1,2,0,18.0,0
4,Female,Loyal Customer,27,Business travel,Business,1550,3,3,3,3,...,2,4,4,5,5,4,2,0,0.0,1
5,Male,Loyal Customer,11,Personal Travel,Eco,526,3,4,3,2,...,4,5,2,5,3,5,4,0,10.0,0


In [165]:
mean_value=train_set["Arrival Delay in Minutes"].mean()
train_set["Arrival Delay in Minutes"].fillna(mean_value,inplace=True)

<ipython-input-165-b5711d91565a>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_set["Arrival Delay in Minutes"].fillna(mean_value,inplace=True)


In [166]:
train_set1=train_set.copy()

In [167]:
labels=train_set1["satisfaction"]
train_set=train_set.drop("satisfaction",axis=1)

In [168]:
train_set.shape

(10000, 22)

In [169]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             10000 non-null  object 
 1   Customer Type                      10000 non-null  object 
 2   Age                                10000 non-null  int64  
 3   Type of Travel                     10000 non-null  object 
 4   Class                              10000 non-null  object 
 5   Flight Distance                    10000 non-null  int64  
 6   Inflight wifi service              10000 non-null  int64  
 7   Departure/Arrival time convenient  10000 non-null  int64  
 8   Ease of Online booking             10000 non-null  int64  
 9   Gate location                      10000 non-null  int64  
 10  Food and drink                     10000 non-null  int64  
 11  Online boarding                    10000 non-null  int64  


In [170]:
train_set_num=train_set.drop(["Gender","Customer Type","Type of Travel","Class"],axis=1)
train_set_string=train_set[["Gender","Customer Type","Type of Travel","Class"]]

In [171]:
ordinal_encoder=OrdinalEncoder()
train_cat_encoder=ordinal_encoder.fit_transform(train_set_string)
train_cat_encoder=pd.DataFrame(train_cat_encoder,columns=["Gender","Customer Type","Type of Travel","Class"])
train_set_num[["Gender","Customer Type","Type of Travel","Class"]]=train_cat_encoder[["Gender","Customer Type","Type of Travel","Class"]].values

In [172]:
standard_scaler=StandardScaler()
train_ready=standard_scaler.fit_transform(train_set_num)
train_ready

array([[-0.42793906, -0.64423433, -0.57254957, ...,  2.21510147,
        -0.64236197,  0.6997325 ],
       [ 0.64608723,  0.19706293,  0.89514992, ..., -0.45144659,
        -0.64236197, -0.90995809],
       [ 0.24332737, -0.35467155, -1.30639932, ..., -0.45144659,
        -0.64236197,  0.6997325 ],
       ...,
       [ 0.4447073 ,  0.2058672 ,  0.16130017, ..., -0.45144659,
        -0.64236197, -0.90995809],
       [ 0.10907408, -1.11477384,  0.89514992, ..., -0.45144659,
        -0.64236197, -0.90995809],
       [ 0.10907408,  0.50619075,  0.16130017, ..., -0.45144659,
        -0.64236197, -0.90995809]])

In [181]:
train_ready_df = pd.DataFrame(train_ready)
train_ready_df.head()
train_ready_df["satisfaction"]=train_set1["satisfaction"].values
columns = train_set1.columns
train_ready_df.columns = columns
train_ready_df

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,-0.427939,-0.644234,-0.572550,-0.042626,-0.561397,0.794100,0.589206,-0.997330,0.387902,0.448356,...,0.508106,-0.578779,0.525416,-0.111703,-0.292833,1.034391,2.215101,-0.642362,0.699732,0
1,0.646087,0.197063,0.895150,-1.352178,0.845966,0.794100,-0.160897,1.220437,0.387902,1.199245,...,-0.286555,1.123263,-0.237272,-0.363861,-0.367855,-0.966752,-0.451447,-0.642362,-0.909958,1
2,0.243327,-0.354672,-1.306399,0.612150,0.845966,0.794100,0.589206,0.481182,-0.374784,-1.804311,...,-1.875876,-2.280821,-0.999961,-0.363861,0.082277,-0.966752,-0.451447,-0.642362,0.699732,0
3,-0.830699,0.313475,0.161300,-0.042626,0.142285,0.014807,-0.911000,-0.997330,-1.137471,-1.053422,...,1.302767,0.272242,-0.999961,-0.363861,-0.367855,-0.966752,-0.451447,-0.642362,-0.909958,1
4,-1.904725,-0.688256,0.161300,0.612150,0.142285,-0.764486,0.589206,-0.258074,0.387902,0.448356,...,-0.286555,1.123263,0.525416,-0.363861,-0.117782,1.034391,-0.451447,1.556755,0.699732,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.981720,-0.158043,-1.306399,0.612150,-1.265079,-0.764486,-0.911000,1.220437,0.387902,0.448356,...,0.508106,0.272242,-0.237272,0.165670,-0.067767,-0.966752,-0.451447,1.556755,0.699732,0
9996,0.981720,0.165759,0.895150,-1.352178,-1.265079,-1.543780,-0.911000,-0.258074,-0.374784,0.448356,...,-1.875876,0.272242,0.525416,0.493476,0.157299,-0.966752,-0.451447,-0.642362,2.309423,0
9997,0.444707,0.205867,0.161300,-0.042626,-0.561397,0.014807,-0.160897,0.481182,1.150589,0.448356,...,-0.286555,0.272242,-0.237272,0.569123,0.582424,1.034391,-0.451447,-0.642362,-0.909958,1
9998,0.109074,-1.114774,0.895150,-1.352178,0.845966,0.794100,1.339309,0.481182,1.150589,-0.302533,...,1.302767,0.272242,1.288105,1.249949,4.883688,-0.966752,-0.451447,-0.642362,-0.909958,1


In [147]:
# Modelni o'rgatish
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(train_ready, labels)

# Bashorat qilish
preds = rf_model.predict(train_ready)
print("Aniqlik: ", accuracy_score(labels, preds))

Aniqlik:  0.9629


In [148]:
pd.DataFrame({"Bashorat":preds,"Asl qiymat":labels})

,Bashorat,Asl qiymat
id,,
1,0,0
2,1,1
3,0,0
4,1,1
5,0,0
...,...,...
9996,0,0
9997,0,0
9998,1,1


In [149]:
cv_scores = cross_val_score(RandomForestClassifier(), train_ready, labels, cv=5)
print("Cross-Validation Aniqliklari: ", cv_scores)
print("O'rtacha CV Aniqlik: ", np.mean(cv_scores))

Cross-Validation Aniqliklari:  [0.951  0.944  0.9515 0.9455 0.9495]
O'rtacha CV Aniqlik:  0.9483


In [ ]:
# An ensemble of Random Forest and LightGBM
ensemble_model = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier()),
    ('lgbm', LGBMClassifier())
], voting='soft')

ensemble_model.fit(train_ready, labels)
preds = ensemble_model.predict(train_ready)
print("Ansambl Aniqligi: ", accuracy_score(labels, preds))

[LightGBM] [Info] Number of positive: 5000, number of negative: 5000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 791
[LightGBM] [Info] Number of data points in the train set: 10000, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Ansambl Aniqligi:  0.9968


# Test set

In [191]:
test_set=pd.read_csv("/content/Aviakompaniya/test_dataset.csv",index_col=0)
test_set.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
id,,,,,,,,,,,,,,,,,,,,,
1,Female,Loyal Customer,25,Personal Travel,Eco,2704,2,2,2,3,...,5,2,4,4,3,5,1,5,152,191.0
2,Female,Loyal Customer,49,Business travel,Business,1623,1,1,1,1,...,4,2,2,2,2,4,2,3,60,52.0
3,Male,Loyal Customer,51,Business travel,Business,338,4,4,4,4,...,5,5,5,5,5,4,5,4,0,0.0
4,Male,Loyal Customer,34,Business travel,Eco Plus,95,0,0,0,3,...,1,5,4,4,1,3,3,5,0,0.0
5,Male,Loyal Customer,34,Personal Travel,Eco,602,4,4,4,3,...,5,5,4,5,4,5,5,5,130,140.0


In [192]:
test_set.shape

(4000, 22)

In [193]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, 1 to 4000
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             4000 non-null   object 
 1   Customer Type                      4000 non-null   object 
 2   Age                                4000 non-null   int64  
 3   Type of Travel                     4000 non-null   object 
 4   Class                              4000 non-null   object 
 5   Flight Distance                    4000 non-null   int64  
 6   Inflight wifi service              4000 non-null   int64  
 7   Departure/Arrival time convenient  4000 non-null   int64  
 8   Ease of Online booking             4000 non-null   int64  
 9   Gate location                      4000 non-null   int64  
 10  Food and drink                     4000 non-null   int64  
 11  Online boarding                    4000 non-null   int64  
 1

In [194]:
mean_value=test_set["Arrival Delay in Minutes"].mean()
test_set["Arrival Delay in Minutes"].fillna(mean_value,inplace=True)

<ipython-input-194-8aa08174a3d9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_set["Arrival Delay in Minutes"].fillna(mean_value,inplace=True)


In [195]:
test_set_num=test_set.drop(["Gender","Customer Type","Type of Travel","Class"],axis=1)
test_set_string=test_set[["Gender","Customer Type","Type of Travel","Class"]]

In [196]:
ordinal_encoder=OrdinalEncoder()
test_cat_encoder=ordinal_encoder.fit_transform(test_set_string)
test_cat_encoder=pd.DataFrame(test_cat_encoder,columns=["Gender","Customer Type","Type of Travel","Class"])
test_set_num[["Gender","Customer Type","Type of Travel","Class"]]=test_cat_encoder[["Gender","Customer Type","Type of Travel","Class"]].values

In [203]:
standard_scaler=StandardScaler()
test_ready=standard_scaler.fit_transform(test_set_num)
test_ready

array([[-1.00949875,  1.4675743 , -0.55639009, ..., -0.47485808,
         1.5946656 ,  0.70492611],
       [ 0.61798677,  0.40000693, -1.28062061, ..., -0.47485808,
        -0.62709072, -0.90541338],
       [ 0.75361056, -0.86902552,  0.89207093, ..., -0.47485808,
        -0.62709072, -0.90541338],
       ...,
       [-0.1279441 , -0.60830601, -0.55639009, ..., -0.47485808,
        -0.62709072,  0.70492611],
       [-1.14512254, -0.23697823,  1.61630145, ..., -0.47485808,
        -0.62709072,  0.70492611],
       [-1.00949875, -0.13032025,  0.16784042, ...,  2.10589235,
        -0.62709072,  0.70492611]])

In [204]:
test_ready_df = pd.DataFrame(test_ready)
test_ready_df.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-1.009499,1.467574,-0.556390,-0.657055,-0.544341,0.015937,-0.961108,-0.265088,1.126410,-1.067904,...,-0.531651,1.324326,-2.261943,1.250539,3.893507,4.947758,-1.006521,-0.474858,1.594666,0.704926
1,0.617987,0.400007,-1.280621,-1.300438,-1.234692,-1.538904,-0.210096,0.475381,0.368268,-1.067904,...,-1.373871,0.529174,-1.417777,-0.265036,1.300368,1.058764,-1.006521,-0.474858,-0.627091,-0.905413
2,0.753611,-0.869026,0.892071,0.629711,0.836359,0.793358,-0.961108,1.215851,1.126410,1.201014,...,1.152788,0.529174,1.114721,0.492752,-0.390810,-0.396111,0.993521,-0.474858,-0.627091,-0.905413
3,-0.399192,-1.109006,-2.004851,-1.943821,-1.925042,0.015937,1.291928,-2.486496,-1.906159,1.201014,...,-2.216090,-0.265978,-0.573611,1.250539,-0.390810,-0.396111,0.993521,-0.474858,-0.627091,2.315266
4,-0.399192,-0.608306,0.892071,0.629711,0.836359,0.015937,0.540916,1.215851,1.126410,1.201014,...,0.310568,1.324326,1.114721,1.250539,3.273408,3.520861,0.993521,-0.474858,1.594666,0.704926


In [207]:
labels=labels[0:4000]
labels

,satisfaction
id,
1,0
2,1
3,0
4,1
5,0
...,...
3996,1
3997,0
3998,0


In [ ]:
#Predict
preds = rf_model.predict(test_ready)
preds=pd.DataFrame(preds)
preds

,0
0,0
1,1
2,1
3,1
4,0
...,...
3995,1
3996,0
3997,0
3998,1


In [219]:
sample_submission=pd.read_csv("/content/Aviakompaniya/sample_submission.csv")
sample_submission["satisfaction"]=preds
sample_submission.to_csv("sample_submission.csv",index=False)
sample_submission

,id,satisfaction
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
3995,3996,1
3996,3997,0
3997,3998,0
3998,3999,1
